In [1]:
from configparser import ConfigParser
import spotipy
import sys
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
import numpy as np

parser = ConfigParser()
parser.read('./spotify_credentials.cfg')

SPOTIPY_CLIENT_ID = parser.get('spotify', 'SPOTIPY_CLIENT_ID')
SPOTIPY_CLIENT_SECRET = parser.get('spotify', 'SPOTIPY_CLIENT_SECRET')

sp = spotipy.Spotify(client_credentials_manager=SpotifyClientCredentials(client_id=SPOTIPY_CLIENT_ID, client_secret=SPOTIPY_CLIENT_SECRET))

In [11]:
input = pd.read_csv('artists.csv', header = None)
artist_ids = []
for link in input[0]:
    
    artist_ids.append(link[32:54])

# Shorten artist_ids
# artist_ids = artist_ids[0:3]

artist_ids

['4tZwfgrHOc3mvqYlEYSvVi',
 '2CIMQHirSU0MQqyYHq0eOx',
 '6nxWCVXbOlEVRexSbLsTer',
 '5K4W6rqBFWDnAN6FQUkS6x',
 '0QWrMNukfcVOmgEU0FEDyD',
 '4LEiUm1SRbFMgfqnQTwUbQ',
 '06HL4z0CvFAxyc27GXpf02',
 '0oeUpvxWsC8bWS6SnpU8b9',
 '3Qy1IxDSU8SLpUUOfbOpxM',
 '5cIc3SBFuBLVxJz58W2tU9',
 '4gzpq5DPGxSnKTe4SA8HAU',
 '1dWEYMPtNmvSVaDNLgB6NV',
 '3iOvXCl6edW5Um0fXEBRXy',
 '26T3LtbuGT1Fu9m0eRq5X3',
 '0pCNk4D3E2xtszsm6hMsWr',
 '3YQKmKGau1PzlVlkL1iodx',
 '3AA28KZvwAUcZuOKwyblJQ',
 '2cFrymmkijnjDg9SS92EPM',
 '7pXu47GoqSYRajmBCjxdD6',
 '4yvcSjfu4PC0CYQyLy4wSq']

In [12]:
albums = []
album_track_ids = []

for artist_id in artist_ids:
    
    album_ids = []
    artist_uri = 'spotify:artist:' + artist_id

    # Add album ids to album_ids list
    results = sp.artist_albums(artist_uri, album_type='album')
    albums.extend(results['items'])
    while results['next']:
        results = sp.next(results)
        albums.extend(results['items'])
    for album in albums:
        album_ids.append(album['id'])

all_tracks = []
for id in album_ids:

    track_ids = []

    results = sp.album_tracks(id)
    tracks = results['items']
    all_tracks.extend(tracks)
    for track in tracks:
        track_ids.append(track['id'])

    album_track_ids.append(track_ids)


assert len(album_track_ids) == len(albums)

len(albums)

325

In [13]:
# all_tracks[0]['track_number']


In [14]:
columns_to_remove = ['analysis_url', 'type', 'uri', 'track_href']
headings = ['danceability',
'energy',
'key',
'loudness',
'mode',
'speechiness',
'acousticness',
'instrumentalness',
'liveness',
'valence',
'tempo',
'id',
'duration_ms',
'time_signature',
'track_title',
'album_title',
'album_artist',
'track_number',
'total_tracks']
data = pd.DataFrame(columns=headings)

# Create DataFrames of features from track ids
i = 0
j = 0
for album in album_track_ids:
    
    length = len(album)
    
    
    for track in album:
        results = sp.audio_features(track)
        if results[0]: 
            features = results[0]
        features_matrix = pd.DataFrame.from_records(features, index=[0])
        
        # Remove unneeded columns
        features_matrix.drop(columns = columns_to_remove, axis = 1, inplace = True)
        # print(features_matrix)

        # Add track title
        track_title = all_tracks[j]['name']
        features_matrix['track_title'] = track_title
        # Add album title
        album_title = albums[i]['name']
        features_matrix['album_title'] = album_title
        # Add artist name
        name = albums[i]['artists'][0]['name']
        features_matrix['album_artist'] = name

        # Add track number and total tracks
        features_matrix['track_number'] = all_tracks[j]['track_number']
        features_matrix['total_tracks'] = length

        # Add album as a row to the data df
        # print(features_matrix,'\n')
        # print(data)
        data = data.append(features_matrix)

        j += 1

    i += 1

# Remove duplicates
data.drop_duplicates(subset=['track_number', 'album_title', 'album_artist'])

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,id,duration_ms,time_signature,track_title,album_title,album_artist,track_number,total_tracks
0,0.788,0.616,9,-10.671,0,0.0337,0.00286,0.549000,0.0952,0.6960,119.037,0dEIca2nhcxDUV8C5QkPYb,275387,4,Give Life Back to Music,Random Access Memories,Daft Punk,1,13
0,0.712,0.500,9,-11.948,0,0.0338,0.82500,0.314000,0.1060,0.6820,90.039,3ctALmweZBapfBdFiIVpji,322147,4,The Game of Love,Random Access Memories,Daft Punk,2,13
0,0.740,0.682,9,-10.420,0,0.0504,0.06920,0.530000,0.3810,0.4100,113.201,0oks4FnzhNp5QPTZtoet7c,544627,4,Giorgio by Moroder,Random Access Memories,Daft Punk,3,13
0,0.702,0.297,10,-14.116,0,0.0317,0.89400,0.485000,0.1030,0.0676,110.116,7Bxv0WL7UC6WwQpk9TzdMJ,228507,4,Within,Random Access Memories,Daft Punk,4,13
0,0.775,0.585,10,-9.516,0,0.0271,0.04220,0.619000,0.0770,0.5180,109.942,2cGxRwrMyEAp8dEbuZaVv6,337560,4,Instant Crush (feat. Julian Casablancas),Random Access Memories,Daft Punk,5,13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,0.651,0.503,1,-10.773,1,0.0381,0.00297,0.007830,0.1120,0.3980,89.993,0Zg4tjgfqw9qzq7lXX2sUM,265893,4,Hazey,ZABA,Glass Animals,7,11
0,0.639,0.347,1,-12.322,0,0.0421,0.40600,0.000268,0.2150,0.2470,72.978,754wVXWrtf1X3ZHQOFFsnR,254640,4,Toes,ZABA,Glass Animals,8,11
0,0.553,0.592,1,-10.205,1,0.1130,0.06920,0.014800,0.3360,0.1890,155.944,6BtC99zhdbWXPGgEOXTTvw,245547,4,Wyrd,ZABA,Glass Animals,9,11
0,0.530,0.275,2,-13.868,0,0.0349,0.46000,0.000011,0.0729,0.0698,133.973,4Qw1VBxYRX2ieU9CADu8Co,271907,4,Cocoa Hooves,ZABA,Glass Animals,10,11


In [15]:
data['order'] = data['track_number']/data['total_tracks']

data.to_csv('./data/data.csv')
data

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,id,duration_ms,time_signature,track_title,album_title,album_artist,track_number,total_tracks,order
0,0.788,0.616,9,-10.671,0,0.0337,0.00286,0.549000,0.0952,0.6960,119.037,0dEIca2nhcxDUV8C5QkPYb,275387,4,Give Life Back to Music,Random Access Memories,Daft Punk,1,13,0.076923
0,0.712,0.500,9,-11.948,0,0.0338,0.82500,0.314000,0.1060,0.6820,90.039,3ctALmweZBapfBdFiIVpji,322147,4,The Game of Love,Random Access Memories,Daft Punk,2,13,0.153846
0,0.740,0.682,9,-10.420,0,0.0504,0.06920,0.530000,0.3810,0.4100,113.201,0oks4FnzhNp5QPTZtoet7c,544627,4,Giorgio by Moroder,Random Access Memories,Daft Punk,3,13,0.230769
0,0.702,0.297,10,-14.116,0,0.0317,0.89400,0.485000,0.1030,0.0676,110.116,7Bxv0WL7UC6WwQpk9TzdMJ,228507,4,Within,Random Access Memories,Daft Punk,4,13,0.307692
0,0.775,0.585,10,-9.516,0,0.0271,0.04220,0.619000,0.0770,0.5180,109.942,2cGxRwrMyEAp8dEbuZaVv6,337560,4,Instant Crush (feat. Julian Casablancas),Random Access Memories,Daft Punk,5,13,0.384615
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,0.530,0.275,2,-13.868,0,0.0349,0.46000,0.000011,0.0729,0.0698,133.973,0tQ3UNjWYzB53QnGzLAqVB,271907,4,Cocoa Hooves,ZABA (Deluxe),Glass Animals,10,14,0.714286
0,0.490,0.495,4,-12.252,0,0.0338,0.17300,0.000000,0.0794,0.2300,148.967,6TvhhNSHNbM7snvQNtBJyk,263667,4,JDNT,ZABA (Deluxe),Glass Animals,11,14,0.785714
0,0.767,0.355,11,-10.031,1,0.0445,0.14600,0.000000,0.2980,0.2880,119.992,37USlegCLu0v62VVacWNAI,161893,4,Love Lockdown - Bonus Track,ZABA (Deluxe),Glass Animals,12,14,0.857143
0,0.743,0.379,1,-13.629,1,0.0784,0.20700,0.196000,0.1110,0.1830,127.030,3KQpqdGO6dJNLvTwZILfvI,412942,4,Pools - Jackson And His Computer Band Remix,ZABA (Deluxe),Glass Animals,13,14,0.928571


In [ ]:
# Live
sp.audio_features('2RpXUwCytDFqZSHl6kIquB')

[{'danceability': 0.26,
  'energy': 0.99,
  'key': 7,
  'loudness': -3.498,
  'mode': 1,
  'speechiness': 0.128,
  'acousticness': 0.00202,
  'instrumentalness': 0.00416,
  'liveness': 0.779,
  'valence': 0.0885,
  'tempo': 128.29,
  'type': 'audio_features',
  'id': '2RpXUwCytDFqZSHl6kIquB',
  'uri': 'spotify:track:2RpXUwCytDFqZSHl6kIquB',
  'track_href': 'https://api.spotify.com/v1/tracks/2RpXUwCytDFqZSHl6kIquB',
  'analysis_url': 'https://api.spotify.com/v1/audio-analysis/2RpXUwCytDFqZSHl6kIquB',
  'duration_ms': 349733,
  'time_signature': 4}]

In [ ]:
# Remastered
sp.audio_features('0xzhryP1AoHUazYdJ5rj3B')

[{'danceability': 0.46,
  'energy': 0.922,
  'key': 2,
  'loudness': -7.388,
  'mode': 1,
  'speechiness': 0.264,
  'acousticness': 0.0449,
  'instrumentalness': 0.000252,
  'liveness': 0.34,
  'valence': 0.328,
  'tempo': 119.961,
  'type': 'audio_features',
  'id': '0xzhryP1AoHUazYdJ5rj3B',
  'uri': 'spotify:track:0xzhryP1AoHUazYdJ5rj3B',
  'track_href': 'https://api.spotify.com/v1/tracks/0xzhryP1AoHUazYdJ5rj3B',
  'analysis_url': 'https://api.spotify.com/v1/audio-analysis/0xzhryP1AoHUazYdJ5rj3B',
  'duration_ms': 311027,
  'time_signature': 4}]

In [ ]:
# Non-remastered (original)
sp.audio_features('4BB3Itr3YETOKFURnC28Ld')

[{'danceability': 0.472,
  'energy': 0.859,
  'key': 2,
  'loudness': -11.866,
  'mode': 1,
  'speechiness': 0.181,
  'acousticness': 0.0793,
  'instrumentalness': 0.000202,
  'liveness': 0.645,
  'valence': 0.322,
  'tempo': 119.989,
  'type': 'audio_features',
  'id': '4BB3Itr3YETOKFURnC28Ld',
  'uri': 'spotify:track:4BB3Itr3YETOKFURnC28Ld',
  'track_href': 'https://api.spotify.com/v1/tracks/4BB3Itr3YETOKFURnC28Ld',
  'analysis_url': 'https://api.spotify.com/v1/audio-analysis/4BB3Itr3YETOKFURnC28Ld',
  'duration_ms': 310667,
  'time_signature': 4}]